In [13]:
from nltk.stem.porter import PorterStemmer
from ast import literal_eval

# def cal_lasting(elem):
    

def tokenize(text):
    stemmeren = PorterStemmer()
    lowercase = [tkn.lower() for tkn in text]
    
    stop_rm = [wd for wd in lowercase if not wd in STOPWORDSEN]
    stem = [stemmeren.stem(wd) for wd in stop_rm]
    return [j for j in stop_rm if j]

# load stop words list
stopWordsSet = sc.textFile("stop-word-list.txt").collect()
STOPWORDSEN = set(stopWordsSet)

with open('data_keywords.txt','r') as f:
    fd = f.read()
res = literal_eval(fd)
res = tokenize(res)
length = len(res)
# print(length)
# print(res)
    

def dcTransformMapper(elem):
    if elem == 'False':
        return [0]
    else:
        return [1]
    
def splitMapper(elem):
    return htf.transform(elem)
    
from pyspark.sql import Row
from pyspark.ml.linalg import Vectors,DenseVector
import numpy as np
# from pyspark.mllib.feature import HashingTF
# from pyspark.mllib.feature import IDF
# from pyspark.mllib.regression import LabeledPoint
from pyspark.ml.feature import HashingTF,IDF,Tokenizer,RegexTokenizer


# def f(x):
#     rel = {}
#     rel['final_status'] = str(x[8])
#     rel['features'] = Vectors.dense(float(x[0]),x[1].toArray(),float(x[2]),float(x[3]),float(x[4]),float(x[5]),float(x[6]),float(x[7]))
#     return rel

import csv
# remove first row(header)
trainKickstarter = sc.textFile("funding-successful-projects-on-kickstarter/train.csv")
trainHeader = trainKickstarter.first()
trainKickstarter = trainKickstarter.filter(lambda line: line != trainHeader).mapPartitions(lambda x: csv.reader(x))
# print(trainKickstarter.take(10))

# Data Transformations and filter lines with empty strings
trainKickstarter=trainKickstarter.map(lambda line: line[3:4]+line[4:5]+dcTransformMapper(line[5])+line[8:14])
# print(trainKickstarter.take(10))

trainKickstarter = spark.createDataFrame(trainKickstarter).toDF('goal','keywords','d_c','deadline','state_change','created_at', 'launched_at','backers','final_status')
trainKickstarter.show(10)

reTokenizer = RegexTokenizer(inputCol="keywords", outputCol="words", pattern="\\W")
wordsData = reTokenizer.transform(trainKickstarter)

hashingTF = HashingTF(inputCol="words", outputCol="rawFeatures", numFeatures=20390)
featurizedData = hashingTF.transform(wordsData)

idf = IDF(inputCol="rawFeatures", outputCol="features")
idfModel = idf.fit(featurizedData)

trainKickstarter = idfModel.transform(featurizedData)
trainKickstarter.select("final_status", "features").show(10)

# htf = HashingTF(length+200)

# hashedData = trainKickstarter.map(lambda line: line[0:1]+htf.transform(line[1]))
# print(hashedData.take(10))
# DenseVectors = hashedData.map(lambda vector: DenseVector(vector.toArray()))
# print(DenseVectors.take(10))
# print(trainKickstarter.take(10))

# trainKickstarter=trainKickstarter.map(lambda p: Row(**f(p))).toDF()
# print(trainKickstarter)

# print(trainKickstarter.show(10))



+----+--------------------+---+----------+------------+----------+-----------+-------+------------+
|goal|            keywords|d_c|  deadline|state_change|created_at|launched_at|backers|final_status|
+----+--------------------+---+----------+------------+----------+-----------+-------+------------+
|  20| drawing-for-dollars|  1|1241333999|  1241334017|1240600507| 1240602723|      3|           1|
| 300|sponsor-dereck-bl...|  1|1242429000|  1242432018|1240960224| 1240975592|      2|           0|
|  30|        mr-squiggles|  1|1243027560|  1243027818|1242163613| 1242164398|      0|           0|
| 500|help-me-write-my-...|  1|1243555740|  1243556121|1240963795| 1240966730|     18|           1|
|2000|support-casting-m...|  1|1243769880|  1243770317|1241177914| 1241180541|      1|           0|
| 700|        daily-digest|  1|1243815600|  1243816219|1241050799| 1241464468|     14|           0|
| 250|igoozex-free-ipho...|  1|1243872000|  1243872028|1241725172| 1241736308|      2|           0|


In [3]:
# RandomForest
from pyspark.ml import Pipeline
from pyspark.ml.feature import IndexToString,StringIndexer,VectorIndexer
from pyspark.ml.classification import RandomForestClassifier
import time

startTime = time.time()

# Index labels, adding metadata to the label column.
# Fit on whole dataset to include all labels in index.
labelIndexer = StringIndexer(inputCol="final_status", outputCol="indexedLabel").fit(trainKickstarter)
# labelIndexer = StringIndexer().setInputCol("label").setOutputCol("indexedLabel").fit(trainTitanic)

# Automatically identify categorical features, and index them.
featureIndexer = VectorIndexer(inputCol="features", outputCol="indexedFeatures").fit(trainKickstarter)
# featureIndexer = VectorIndexer().setInputCol("features").setOutputCol("indexedFeatures").fit(trainTitanic)


# Split the data into training and test sets (30% held out for testing)
(trainingData, testData) = trainKickstarter.randomSplit([0.7, 0.3])

# Train a RandomForest model.
rf = RandomForestClassifier(labelCol="indexedLabel", featuresCol="indexedFeatures", numTrees=10)

# Convert indexed labels back to original labels.
labelConverter = IndexToString(inputCol="prediction", outputCol="predictedLabel",labels=labelIndexer.labels)


# Chain indexers and forest in a Pipeline
Pipeline =  Pipeline().setStages([labelIndexer, featureIndexer, rf, labelConverter])
# Train model.  This also runs the indexers.
PipelineModel = Pipeline.fit(trainingData)

endTime = time.time()
executeTime = endTime - startTime
print ('Execution Time For Random Forest = %s' % str(executeTime) + ' Seconds\n')

# Make predictions.
Predictions = PipelineModel.transform(testData)
Predictions.select("predictedLabel", "final_status").show(10)


preRel = Predictions.select("predictedLabel", "final_status", "features", "probability").collect()

print('''final_status --> prob = [prob1,prob2], predicted_final_status''')
for item in preRel:
    print(str(item['final_status'])+' -->prob= '+str(item['probability'])+',predicted_final_status: '+str(item['predictedLabel']))
 


Execution Time For Random Forest = 130.3072440624237 Seconds

+--------------+------------+
|predictedLabel|final_status|
+--------------+------------+
|             0|           0|
|             0|           1|
|             0|           1|
|             0|           1|
|             0|           1|
|             0|           1|
|             0|           1|
|             0|           1|
|             0|           1|
|             0|           0|
+--------------+------------+
only showing top 10 rows

final_status --> prob = [prob1,prob2], predicted_final_status
0 -->prob= [0.6840975846120083,0.3159024153879917],predicted_final_status: 0
1 -->prob= [0.6840975846120083,0.3159024153879917],predicted_final_status: 0
1 -->prob= [0.6811459559998384,0.31885404400016154],predicted_final_status: 0
1 -->prob= [0.6811459559998384,0.31885404400016154],predicted_final_status: 0
1 -->prob= [0.6811459559998384,0.31885404400016154],predicted_final_status: 0
1 -->prob= [0.6840975846120083,0.315902415

1 -->prob= [0.6736047019120486,0.3263952980879514],predicted_final_status: 0
0 -->prob= [0.6840975846120083,0.3159024153879917],predicted_final_status: 0
0 -->prob= [0.6073052835027964,0.3926947164972035],predicted_final_status: 0
0 -->prob= [0.6840975846120083,0.3159024153879917],predicted_final_status: 0
0 -->prob= [0.6840975846120083,0.3159024153879917],predicted_final_status: 0
1 -->prob= [0.6840975846120083,0.3159024153879917],predicted_final_status: 0
1 -->prob= [0.6840975846120083,0.3159024153879917],predicted_final_status: 0
1 -->prob= [0.6736047019120486,0.3263952980879514],predicted_final_status: 0
0 -->prob= [0.7013788186532826,0.2986211813467175],predicted_final_status: 0
0 -->prob= [0.6840975846120083,0.3159024153879917],predicted_final_status: 0
0 -->prob= [0.6811459559998384,0.31885404400016154],predicted_final_status: 0
1 -->prob= [0.6625071760842408,0.33749282391575924],predicted_final_status: 0
1 -->prob= [0.6625071760842408,0.33749282391575924],predicted_final_status

0 -->prob= [0.6811459559998384,0.31885404400016154],predicted_final_status: 0
0 -->prob= [0.6840975846120083,0.3159024153879917],predicted_final_status: 0
0 -->prob= [0.6840975846120083,0.3159024153879917],predicted_final_status: 0
1 -->prob= [0.6840975846120083,0.3159024153879917],predicted_final_status: 0
1 -->prob= [0.6811459559998384,0.31885404400016154],predicted_final_status: 0
0 -->prob= [0.6840975846120083,0.3159024153879917],predicted_final_status: 0
0 -->prob= [0.6840975846120083,0.3159024153879917],predicted_final_status: 0
0 -->prob= [0.6840975846120083,0.3159024153879917],predicted_final_status: 0
1 -->prob= [0.6840975846120083,0.3159024153879917],predicted_final_status: 0
0 -->prob= [0.6840975846120083,0.3159024153879917],predicted_final_status: 0
0 -->prob= [0.6811459559998384,0.31885404400016154],predicted_final_status: 0
0 -->prob= [0.6840975846120083,0.3159024153879917],predicted_final_status: 0
0 -->prob= [0.6840975846120083,0.3159024153879917],predicted_final_status

0 -->prob= [0.6840975846120083,0.3159024153879917],predicted_final_status: 0
0 -->prob= [0.6840975846120083,0.3159024153879917],predicted_final_status: 0
1 -->prob= [0.6840975846120083,0.3159024153879917],predicted_final_status: 0
1 -->prob= [0.6689540182226466,0.3310459817773535],predicted_final_status: 0
0 -->prob= [0.6840975846120083,0.3159024153879917],predicted_final_status: 0
0 -->prob= [0.6840975846120083,0.3159024153879917],predicted_final_status: 0
0 -->prob= [0.6725890761495671,0.32741092385043297],predicted_final_status: 0
1 -->prob= [0.6840975846120083,0.3159024153879917],predicted_final_status: 0
0 -->prob= [0.6840975846120083,0.3159024153879917],predicted_final_status: 0
0 -->prob= [0.6840975846120083,0.3159024153879917],predicted_final_status: 0
0 -->prob= [0.6840975846120083,0.3159024153879917],predicted_final_status: 0
0 -->prob= [0.6811459559998384,0.31885404400016154],predicted_final_status: 0
0 -->prob= [0.6755407047617369,0.3244592952382631],predicted_final_status:

0 -->prob= [0.6811459559998384,0.31885404400016154],predicted_final_status: 0
0 -->prob= [0.6811459559998384,0.31885404400016154],predicted_final_status: 0
0 -->prob= [0.6811459559998384,0.31885404400016154],predicted_final_status: 0
1 -->prob= [0.6811459559998384,0.31885404400016154],predicted_final_status: 0
0 -->prob= [0.6811459559998384,0.31885404400016154],predicted_final_status: 0
0 -->prob= [0.6811459559998384,0.31885404400016154],predicted_final_status: 0
1 -->prob= [0.6811459559998384,0.31885404400016154],predicted_final_status: 0
0 -->prob= [0.6811459559998384,0.31885404400016154],predicted_final_status: 0
1 -->prob= [0.6811459559998384,0.31885404400016154],predicted_final_status: 0
0 -->prob= [0.6840975846120083,0.3159024153879917],predicted_final_status: 0
0 -->prob= [0.6840975846120083,0.3159024153879917],predicted_final_status: 0
0 -->prob= [0.6840975846120083,0.3159024153879917],predicted_final_status: 0
1 -->prob= [0.6840975846120083,0.3159024153879917],predicted_final_

...

0 -->prob= [0.6536844222447775,0.3463155777552226],predicted_final_status: 0
0 -->prob= [0.6811459559998384,0.31885404400016154],predicted_final_status: 0
0 -->prob= [0.6840975846120083,0.3159024153879917],predicted_final_status: 0
1 -->prob= [0.6840975846120083,0.3159024153879917],predicted_final_status: 0
0 -->prob= [0.6840975846120083,0.3159024153879917],predicted_final_status: 0
0 -->prob= [0.6520219440167303,0.34797805598326975],predicted_final_status: 0
0 -->prob= [0.6840975846120083,0.3159024153879917],predicted_final_status: 0
0 -->prob= [0.6811459559998384,0.31885404400016154],predicted_final_status: 0
0 -->prob= [0.6840975846120083,0.3159024153879917],predicted_final_status: 0
0 -->prob= [0.6840975846120083,0.3159024153879917],predicted_final_status: 0
0 -->prob= [0.6840975846120083,0.3159024153879917],predicted_final_status: 0
1 -->prob= [0.6671267757737243,0.33287322422627563],predicted_final_status: 0
1 -->prob= [0.6840975846120083,0.3159024153879917],predicted_final_statu

1 -->prob= [0.6840975846120083,0.3159024153879917],predicted_final_status: 0
0 -->prob= [0.6840975846120083,0.3159024153879917],predicted_final_status: 0
0 -->prob= [0.6840975846120083,0.3159024153879917],predicted_final_status: 0
0 -->prob= [0.6840975846120083,0.3159024153879917],predicted_final_status: 0
0 -->prob= [0.6840975846120083,0.3159024153879917],predicted_final_status: 0
0 -->prob= [0.6840975846120083,0.3159024153879917],predicted_final_status: 0
1 -->prob= [0.6840975846120083,0.3159024153879917],predicted_final_status: 0
1 -->prob= [0.6840975846120083,0.3159024153879917],predicted_final_status: 0
1 -->prob= [0.6811459559998384,0.31885404400016154],predicted_final_status: 0
0 -->prob= [0.6587036716117312,0.34129632838826873],predicted_final_status: 0
0 -->prob= [0.6840975846120083,0.3159024153879917],predicted_final_status: 0
1 -->prob= [0.6840975846120083,0.3159024153879917],predicted_final_status: 0
0 -->prob= [0.6840975846120083,0.3159024153879917],predicted_final_status:

0 -->prob= [0.6840975846120083,0.3159024153879917],predicted_final_status: 0
1 -->prob= [0.6811459559998384,0.31885404400016154],predicted_final_status: 0
1 -->prob= [0.6840975846120083,0.3159024153879917],predicted_final_status: 0
0 -->prob= [0.6840975846120083,0.3159024153879917],predicted_final_status: 0
1 -->prob= [0.6811459559998384,0.31885404400016154],predicted_final_status: 0
0 -->prob= [0.6840975846120083,0.3159024153879917],predicted_final_status: 0
0 -->prob= [0.6840975846120083,0.3159024153879917],predicted_final_status: 0
0 -->prob= [0.6840975846120083,0.3159024153879917],predicted_final_status: 0
0 -->prob= [0.6840975846120083,0.3159024153879917],predicted_final_status: 0
0 -->prob= [0.6625071760842408,0.33749282391575924],predicted_final_status: 0
1 -->prob= [0.6840975846120083,0.3159024153879917],predicted_final_status: 0
0 -->prob= [0.6840975846120083,0.3159024153879917],predicted_final_status: 0
0 -->prob= [0.6840975846120083,0.3159024153879917],predicted_final_status

0 -->prob= [0.6840975846120083,0.3159024153879917],predicted_final_status: 0
0 -->prob= [0.6840975846120083,0.3159024153879917],predicted_final_status: 0
0 -->prob= [0.6840975846120083,0.3159024153879917],predicted_final_status: 0
0 -->prob= [0.6840975846120083,0.3159024153879917],predicted_final_status: 0
0 -->prob= [0.6840975846120083,0.3159024153879917],predicted_final_status: 0
0 -->prob= [0.6840975846120083,0.3159024153879917],predicted_final_status: 0
0 -->prob= [0.6840975846120083,0.3159024153879917],predicted_final_status: 0
0 -->prob= [0.6840975846120083,0.3159024153879917],predicted_final_status: 0
1 -->prob= [0.6811459559998384,0.31885404400016154],predicted_final_status: 0
0 -->prob= [0.6921003476609567,0.30789965233904326],predicted_final_status: 0
0 -->prob= [0.7013788186532826,0.2986211813467175],predicted_final_status: 0
0 -->prob= [0.6840975846120083,0.3159024153879917],predicted_final_status: 0
1 -->prob= [0.6840975846120083,0.3159024153879917],predicted_final_status:

0 -->prob= [0.6840975846120083,0.3159024153879917],predicted_final_status: 0
1 -->prob= [0.6616540357072642,0.33834596429273567],predicted_final_status: 0
0 -->prob= [0.6840975846120083,0.3159024153879917],predicted_final_status: 0
0 -->prob= [0.6840975846120083,0.3159024153879917],predicted_final_status: 0
0 -->prob= [0.6840975846120083,0.3159024153879917],predicted_final_status: 0
0 -->prob= [0.6811459559998384,0.31885404400016154],predicted_final_status: 0
0 -->prob= [0.6840975846120083,0.3159024153879917],predicted_final_status: 0
1 -->prob= [0.6840975846120083,0.3159024153879917],predicted_final_status: 0
0 -->prob= [0.6811459559998384,0.31885404400016154],predicted_final_status: 0
0 -->prob= [0.6840975846120083,0.3159024153879917],predicted_final_status: 0
0 -->prob= [0.6840975846120083,0.3159024153879917],predicted_final_status: 0
1 -->prob= [0.6540850854113981,0.3459149145886019],predicted_final_status: 0
0 -->prob= [0.6840975846120083,0.3159024153879917],predicted_final_status

In [5]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
# Select (prediction, true label) and compute test error
evaluator = MulticlassClassificationEvaluator(labelCol="indexedLabel", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(Predictions)
print("Test Error = %g" % (1.0 - accuracy))
print("Test Accuracy = %g" % accuracy)

rfModel = PipelineModel.stages[2]
print(rfModel)  # summary only

# # Transfer final_status(dataframe) into RDD
# ori_result = testData.rdd.map(lambda p: float(p.final_status))
# pre_result = Predictions.rdd.map(lambda p: float(p.final_status))
# # print(ori_result)  #RDD
# # print(pre_result)  #RDD
# labelsAndPredictions = ori_result.zip(pre_result)
# print(labelsAndPredictions.take(10))

# # first metrics
# from pyspark.mllib.evaluation import BinaryClassificationMetrics
# metrics = BinaryClassificationMetrics(labelsAndPredictions)
 
# # Area under precision-recall curve
# print("Area under PR = %s" % metrics.areaUnderPR)
 
# # Area under ROC curve
# print("Area under ROC = %s" % metrics.areaUnderROC)

# # second metrics
# accuracyTest = 1.0 * labelsAndPredictions.filter(lambda xv: xv[0] == xv[1]).count() / testData.count()
# print ("Test Accuracy For Random Forest = "  + str(accuracyTest) + "\n")


Test Error = 0.32096
Test Accuracy = 0.67904
RandomForestClassificationModel (uid=RandomForestClassifier_46438ec8bc411293d260) with 10 trees
